# BC Data Workshop - Video Compression Project

The quality of video is affected by the video capture, compression,  transmission, and also by factors on the audience end. We present a non-exhaustive tour of some important features.


** Resolution : number of pixels displayed and/or captured. **   
- 720 x 480 (or 480p) "SD"
- 3840 x 2160 (or 2160p) "4K"  
- The content of the scene may dictate the necessary resolution, e.g. the level of detail.

** Framerate: rate at which frames are displayed. **  
- Increasing the framerate (fps) will result in a smoother video at a given resolution. 
- Standards for framerate vary from media to media. 
- Content of the scene may require a higher framerate 


** Bitrate: Size of video file streaming per second. **

- trade-off between quality and storage/cost (also possible lags/buffering if streaming)
- Need to ensure that most user's have adequate bandwith. 
- Higher resolution &/or framerate requires higher bitrate

In general, we want to be able to reduce bitrate while maintaining quality



## Video Compression

** Frame-by-Frame encoding **   
- Compress each individual frame 
- e.g. JPEG



** Temporal encoding **  

- Attempt to only store the changes between frames

- e.g. H.264 and MPEG-4


Basic framework: we encode frames as one of three frame type: Intra(I)-frames, Predicted(P)-frames and Bidirectional(B)-frames. 
- I-Frames : complete encoded images 
- P-Frames : coded with reference to an earlier I- or P-frame. 
- B-Frames : contain information on the changes between previous and following frames.

On average, using H264, we can reduce bitrate by ~50% for fixed quality. 

# A first look at the data

Our goal will be to create a model which predicts bitrate given various camera and scene features. 

Bitrate measurements were taken after capturing and compressing video taken under different test conditions (listed below). 

The data was provided by Roger Donaldson (Midvale Applied Mathematics).


### Camera Features (from 8 different cameras)
- Flicker: camera cuts out light flicker, depending on the region
- KeyFrame: number of P- and B-frames between I-frames (minus 1)
- ImageRate: framerate (fps)
- Quality: compression control, low number is best quality
- Nonlinear: single or multi-exposure frames (HDR)


- Mode: high performance or standard modes
- Compression: additional compression mode
- KpbsLimit: user set maximum bitrate
- Primary Resolution: highest resolution stream
- Secondary Resolution: lower resolution stream
- Tertiary Resolution: lowest resolution stream

### Scene Features
- Test: type of test (Base, Idle, Compression, or HDR)
- Motion: amount of motion in scene, classified as high, medium or none
- Detail: the amount of detial in scene, classified as high, medium, or low

### Measurements
- CollectSeconds: time of data collection
- WaitSeconds: time between measurements
- TotalBytes: total number of bytes transmitted to the network
- Primary Bitrate: mean bitrate for primary stream
- Secondary Bitrate: mean bitrate for secondary stream
- Tertiary Bitrate: mean bitrate for tertiary stream

# Preprocessing and Visualization

In [ ]:
datadir = '/home/nmulberry/data/1-Donaldson/'
datafile = 'AllData.csv'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 50
plt.rcParams['font.size'] = 16

In [ ]:
# Preprocessing data by Aaron
def checkColumn(df, colNum):
    """
    Used in throwAwayUnchanged
    """
    return np.all(df.iloc[0, colNum] == df.iloc[1:, colNum])

def removeUnwantedCols(data):
    drop_cols = ['Flicker','Index','Sharpening','TotalBytes','Status','Message',
                 'TertiaryBitsPerSecond','TertiaryResolution']
    for c in drop_cols:
        data = data.drop(c,axis=1)
    data.groupby('Mode').get_group(0)
    data = removeUnchanging(data)
    return data

def removeUnchanging(data):
    idxUnhelpful = [j for j in range(data.columns.size) if checkColumn(data,j)]
    data = data.drop(data.columns[idxUnhelpful],axis=1)    
    return data

def fixMiscValues(data):
    data = data.fillna({'TertiaryResolution' : 'NaN'})
    data = data.replace('-', value=0)
    data['SecondaryBitsPerSecond'] = data['SecondaryBitsPerSecond'].astype(np.float64)
    #data['TertiaryBitsPerSecond'] = data['TertiaryBitsPerSecond'].astype(np.float64)
    return data

def appendFeatureName(data):
    cols = ['Compression','Test','Detail','Motion','CameraName','PrimaryResolution','SecondaryResolution']
    for c in cols:
        data[c] = data[c].apply(lambda x: c + '_' + x)
    return data

def logTransformColumn(data, colname):
    """
    Tailor-made for the Midvale data. 
    log-transforms the columns pertaining to bit-rate.
    """
    logBytes = data[colname]
    logBytes = logBytes.replace(0., np.nan).apply(lambda x: np.log10(x))
    logBytes = logBytes.dropna(how='all')
    return data.assign(**{'log'+logBytes.name: logBytes})

def addLogCols(data):
    for columnName in ['PrimaryBitsPerSecond', 
                       'SecondaryBitsPerSecond']:
        data = logTransformColumn(data, columnName)
    return data

def preProcess(data):
    data = removeUnwantedCols(data)
    data = fixMiscValues(data)
    data = addLogCols(data)
    data = appendFeatureName(data)
    return data


In [ ]:
data = pd.read_csv(datadir + datafile)
data = preProcess(data)
data.head()

In [ ]:
numerical_names = ['Keyframe', 'ImageRate', 'Quality', 'KbpsLimit']
numerical = data.filter(items=numerical_names)

categ_names = ['PrimaryResolution', 'SecondaryResolution', 'Detail', 'Test', 
               'Motion','CameraName','Nonlinear','Mode','Compression']
categ = data.filter(items=categ_names)

response_names = ['logPrimaryBitsPerSecond', 'logSecondaryBitsPerSecond']
responses = data.filter(items=response_names)

excluded_names = [j for j in data.columns if (j not in categ_names and j not in numerical_names and j not in response_names and j not in ['PrimaryBitsPerSecond','SecondaryBitsPerSecond']) ]
print('Included numerical features',numerical_names)
print('Included categorical features',categ_names)
print('Excluded features',excluded_names)

# Data processing by Aaron
def categDF(data):
    categoricals = data.select_dtypes(include=['object'])
    #categoricals = categoricals.drop('Message', axis=1)
    return categoricals

def setUpCategs(data, sparse=False):
    from sklearn.preprocessing import LabelEncoder, OneHotEncoder
    lb = LabelEncoder()
    oh = OneHotEncoder()
    categoricals = categDF(data)
    categoricals = categoricals.apply(lb.fit_transform)
    categoricals = oh.fit_transform(categoricals)
    if not sparse:
        categoricals = categoricals.toarray()
    return categoricals

def unencodeOneHotLabelling(ohEnc, oh, lbl):
    return lbl.inverse_transform(oh.active_features_)[np.argmax(ohEnc, axis=-1)]

categoricals = setUpCategs(categ)
categ_data = categDF(categ)

In [ ]:
logBytes = data.loc[:, ['PrimaryBitsPerSecond', 'SecondaryBitsPerSecond']]
logBytes = logBytes.replace(0., np.nan).apply(lambda x: np.log(x)).dropna(how='all')
logBytes.hist(figsize=(10,6));

In [ ]:
def logTransformColumn(df, colname):
    """
    Tailor-made for the Midvale data. 
    log-transforms the columns pertaining to bit-rate.
    """
    logBytes = data[colname]
    logBytes = logBytes.replace(0., np.nan).apply(lambda x: np.log(x))
    logBytes = logBytes.dropna(how='all')
    return df.assign(**{'log'+logBytes.name: logBytes})

def logTransformBytes(df):
    for columnName in ['PrimaryBitsPerSecond', 
                       'SecondaryBitsPerSecond']:
        df = logTransformColumn(df, columnName)
    return df

data = logTransformBytes(data)

def hist_colVals(X, **kwargs):
    """
    X : a categorical column of a data frame
    """
    # Check if not categorical
    #
    #
    # get value counts
    vc = X.value_counts()
    n = vc.shape[0]
    xrange = np.arange(n)
    plt.bar(xrange, vc.values, **kwargs)
    plt.xticks(xrange, vc.index.tolist(), rotation=90)
    return
    
    # make a histogram of these
# (these are the *useful* categories for CamerName:A3;Test:Base)
categs = ['PrimaryResolution', 'SecondaryResolution', 
          'Keyframe', 'ImageRate', 'Quality',
          'Detail', 'Motion']

C = len(categs)
ncols = 4
nrows = np.int(np.ceil(C/5))
figwidth = 20
figheight = np.int(np.min([np.ceil(20/ncols*nrows), 20]))


In [ ]:

fig, axes = plt.subplots(nrows, ncols, figsize=(figwidth, figheight))
fig.subplots_adjust(hspace=.75)

for j, categ in enumerate(categs):
    plt.subplot(nrows, ncols, j+1)
    hist_colVals(data[categ])
    plt.xlabel(categ, labelpad=20)
for j in range(C, nrows*ncols):
    plt.subplot(nrows, ncols, j+1)
    plt.axis('off')
    


In [ ]:
data['logPrimaryBitsPerSecond'].hist(bins=1000, cumulative=True)

# Regression Models

In [ ]:
from sklearn.model_selection import train_test_split

def getCategNames(df):
    return np.concatenate([np.unique(df[col].values) for col in df.columns])

def unencodeOneHot(ohArr, df):
    categNames = getCategNames(df)
    return [categNames[np.where(ohArr)[1]][j::2] for j in range(2)]

categ_names = getCategNames(categ_data)
data_names = np.concatenate((numerical_names, categ_names))

scaler = StandardScaler()
X = np.hstack((scaler.fit_transform(numerical.values), categoricals))
y = responses.values
y_pbps = y[:, 0]
nonnanrows = [not np.isnan(y_pbps[j]) for j in range(y_pbps.shape[0])]
X_nonnan = X[nonnanrows, :]
y_nonnan = y_pbps[nonnanrows]

In [ ]:

X_train, X_test,y_train,y_test = train_test_split(X_nonnan,y_nonnan)
ntrain = X_train.shape[0]
ntest = X_test.shape[0]
nfeat = X_train.shape[1]
print('# of training samples {}'.format(ntrain))
print('# of test samples {}'.format(ntest))
print('# of features {}'.format(nfeat))


In [ ]:
# Output statistics for after regression
res_func_inv = lambda x: 10**x
pdeviation = 0.2
def RMSE(output, prediction):
    return np.sqrt(np.mean((res_func_inv(output) - res_func_inv(prediction))**2))
def NRMSE(output, prediction):
    return RMSE(output,prediction)/(np.max(res_func_inv(output))-np.min(res_func_inv(output)))
def percentHits(output,prediction,pdeviation):
    return np.mean( np.abs(res_func_inv(output)-res_func_inv(prediction))<pdeviation*abs(res_func_inv(output)) )

def regressionSummary(method,Xtrain,Ytrain,Xtest,Ytest):
    predict_train = method.predict(Xtrain)
    train_nrmse = NRMSE(Ytrain,predict_train)
    train_hits = percentHits(Ytrain,predict_train,pdeviation)
    Rsq_train = method.score(Xtrain,Ytrain)
    predict_test = method.predict(Xtest)    
    test_nrmse = NRMSE(Ytest,predict_test)
    test_hits = percentHits(Ytest,predict_test,pdeviation)
    Rsq_test = method.score(Xtest,Ytest)
    print('Regression results for ',method.__class__)
    print('------Training data------')
    print('NRMSE = {}'.format(train_nrmse))
    print('Percentage of values with <',pdeviation*100,'percent rel err =',train_hits)
    print('R^2 = {}'.format(Rsq_train))
    print('------Test data----------')
    print('NRMSE = {}'.format(test_nrmse))
    print('Percentage of values with <',pdeviation*100,'percent rel err =',test_hits)
    print('R^2 = {}'.format(Rsq_test))

In [ ]:
# Plain linear regression
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,y_train)

regressionSummary(lr,X_train,y_train,X_test,y_test)

In [ ]:
# Plain elastic net
if ntrain < 5000:
    from sklearn.linear_model import ElasticNetCV
    en = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], n_jobs=-1)
    en.fit(X_train,y_train)
    regressionSummary(en,X_train,y_train,X_test,y_test)

In [ ]:
# Kernel ridge regression - takes a while
from sklearn.kernel_ridge import KernelRidge
kr = KernelRidge(alpha=1,kernel='rbf',gamma=0.1)
kr.fit(X_train,y_train)
regressionSummary(kr,X_train,y_train,X_test,y_test)

In [ ]:
# Random Forest Regression 
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=10)
rf.fit(X_train,y_train)
regressionSummary(rf,X_train,y_train,X_test,y_test)

In [ ]:
# Extra Trees Regression
from sklearn.ensemble import ExtraTreesRegressor
et = ExtraTreesRegressor()
et.fit(X_train,y_train)
regressionSummary(et,X_train,y_train,X_test,y_test)

In [ ]:
# Gradient Boosting - takes a while
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(n_estimators=100)
gb.fit(X_train,y_train)
regressionSummary(gb,X_train,y_train,X_test,y_test)

In [ ]:
# Feature importances for Extra Trees Regressor
importances = et.feature_importances_
std = np.std([tree.feature_importances_ for tree in et.estimators_],axis=0)
indices = np.argsort(importances)[::-1]
dispindex = np.where(importances[indices] < 0.05*importances[indices][0])[0][0]
importances_trunc = importances[indices][:dispindex]
names_trunc = data_names[indices][:dispindex]
std_trunc = std[indices][:dispindex]
plt.figure()
plt.bar(range(dispindex),importances_trunc,color="r",yerr=std_trunc)
plt.xticks(range(dispindex),names_trunc,rotation='vertical')
plt.title('Feature Importance')
plt.show()

In [ ]:
# Look at the cumulative regression performance for each method
p = np.linspace(0.5,0.05,num=10)
def cumul_regress_performance(re,Xtest,Ytest):
    pred = re.predict(Xtest)
    percent = np.zeros(len(p))
    for i in range(len(p)):
        percent[i] = percentHits(Ytest,pred,p[i])
    return percent

regr_methods = [lr,kr,rf,et,gb]
performance = []
for m in regr_methods:
    performance.append(cumul_regress_performance(m,X_test,y_test))
nmethods = len(regr_methods)

In [ ]:
ax = plt.figure(figsize=(10,5))
regr_labels = ['LinReg','KernRidge','RandForest','ExtraTrees','GradBoost']
for i in range(nmethods):
    plt.plot(range(len(p)),100*performance[i],label=regr_labels[i])

plt.xlabel('Relative error rate')
plt.ylabel('% Predicted within rel. err')
plt.xticks(range(len(p)),100*p)
plt.legend()
plt.show()
